In [30]:
import requests
import time
import networkx

The following function will be used to retrieve the transaction hashes from a "block" from the Bitcoin blockchain. Blocks simply hold transaction info from a certain set of timestamps. We will be doing this by using Blockchair's API endpoint throughout this notebook.

In [31]:
def get_block_transactions(height):
    response = requests.get(f"https://api.blockchair.com/bitcoin/dashboards/block/{height}")
    hash_value = response.json()['data'][f'{height}']['transactions']
    print(hash_value)
    return hash_value
    
transactions = get_block_transactions(893211)

['c789c094f68dcfb9516b6c48e39c3dd27914aae7563b6a9a9c656b488f89c5ba', 'afd521f6eb5e7d958dd798f49bbc2d0abbbd42592f1f6f4e24929cae64810d7b', '0ce777ef98e67df2462e75d74fbfaf254b60cbd15d5b52509268b28c16be6b51', '33395802910d8089c27fa15066aac9af3e74479e82b430014165fc683460ec5d', '3c23a71d4b7e2da4c49cd36c90985ac450f70e4a5c692c5deb8d8910d0165abe', '2c5d54cd1ca1c7b695514cf08c468f19281ed2e234793c37e6fd14ff14e4340d', 'b43d37ee446ee6c33664e9e9781dc2165dec0d50bf3e8684cbd0062d83315132', 'f0fc3394851d75e157a24cea43ba04c2b92566c9330944137b563cda07411feb', '95e366bb77052b2b8cbc3d5bbd53dc5c4d7a85b563a09d5c7644f58dea8ef1fd', 'f7cabf97bfcca9b978a38abe172a8b77081383d79e81efa6d3501ae24f876386', '5f1635dbc7df8500bc52fc37b7028370ead792598341767be9c3b1ceddc6c1e9', '940e7a4fd6fb43a72c34b114f6fde0d6915c7a387a40b7e2a2d3ede9e0609a01', 'dd86a5d76364fd0e5c2b93e3ac2064355591dd67364ad06e2cc741fcf8764a3a', 'c1c064d9598a952fd064636b5a689e52dccb2c182a68f4d7120cce7ef2c0e9af', '00036755704b988c526c0f8a310ad13bb176c9c804ed84

Next, we take these transaction hashes and call a second endpoint to get the wallet addresses of the sender (labelled as input), as well as the recipient(s) (labelled output 1, 2, etc). There are situations where the sender will have an output as well, this is when the order has change that is sent back to the sender.

In [32]:

def get_addresse_inputs_and_outputs(trans_hash):
    
    
    try:
        response = requests.get(f"https://api.blockchair.com/bitcoin/dashboards/transaction/{trans_hash}")
        if response.status_code != 200:
            return
        
        json_data = response.json()
    except Exception as e:
        print(f"❌ Failed to parse JSON for {trans_hash}: {e}")
        return

    data_dict = json_data.get('data', {})
    if trans_hash not in data_dict:
        print(f"❌ Transaction hash not found in response: {trans_hash}")
        return

    data = data_dict[trans_hash]
    
    if data.get('transaction', {}).get('is_coinbase'):
        print(f"⛏️ Coinbase transaction skipped: {trans_hash}")
        return

    inputs = data.get('inputs', [])
    outputs = data.get('outputs', [])
    
    if not inputs:
        print(f"⚠️ No inputs found for transaction {trans_hash}")
        return

    if not outputs:
        print(f"⚠️ No outputs found for transaction {trans_hash}")
        return

    input_wallet = inputs[0].get('recipient', 'UNKNOWN_INPUT')
    output_wallets = [output.get('recipient', 'UNKNOWN_OUTPUT') for output in outputs]
    output_values = [output.get('value', 0) / 100000000 for output in outputs]
    

    combined = [input_wallet, *output_values, *output_wallets]
    list_of_wallets.append(combined)    

list_of_wallets = []
for i in transactions[:1000]:
    get_addresse_inputs_and_outputs(i)
    time.sleep(1.5)

print(list_of_wallets)

⛏️ Coinbase transaction skipped: c789c094f68dcfb9516b6c48e39c3dd27914aae7563b6a9a9c656b488f89c5ba
[['3QiETomgUhPu573ZvhXbdofq7y5ocNS1ie', 0.00187043, 2.3588799, 'bc1q2lewcd34manw2f4xw7n6lgjyt6ghx955snd3c0', '3QiETomgUhPu573ZvhXbdofq7y5ocNS1ie'], ['bc1qryhgpmfv03qjhhp2dj8nw8g4ewg08jzmgy3cyx', 0.01447595, 0.00105711, 'bc1qryhgpmfv03qjhhp2dj8nw8g4ewg08jzmgy3cyx', 'bc1qjzfd8lklg9puf85nkasarv25xsufuqadzzg0wxa8g5xlhlql5g4smj8exe'], ['bc1pa6kpdcjc6zgtdpv2kku5jxxfujmmzh0vl5mg56y3eyesgdmj356s42dyal', 2.02875151, 'bc1qsh49g2tf829jwp5md4afx8eg49jk0a9rzn23yp'], ['bc1q452chy76h70tz02pea525qdm8js5zusz5w99en', 0.025, 0.00464744, 'bc1qx8nxcjqlxplqsrjd9s89cjh6lvcyjq3us84cr5', 'bc1q452chy76h70tz02pea525qdm8js5zusz5w99en'], ['bc1p0wwtdmwv47a2xvej32d07hv0ztflrahpej2ewuvmk0x7rj968frs0ku0qd', 7.636e-05, 0.00079, 0.00079, 'bc1p0wwtdmwv47a2xvej32d07hv0ztflrahpej2ewuvmk0x7rj968frs0ku0qd', 'bc1qe7pslj3p0cjr5s0gzy6p03xg9g5ng0mwlyxrcd', 'bc1qe7pslj3p0cjr5s0gzy6p03xg9g5ng0mwlyxrcd'], ['3PbYm5SPCtbac8i71TAW88pvdgCJ

Now that we have our addresses and hashes, lets write it to a file!

In [33]:
import csv

with open('walletswithweight.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Input', 'Value1', 'Value2', '...', 'Output1', 'Output2', '...'])
    for row in list_of_wallets:
        writer.writerow(row)